# Backtest Notebook (Synthetic)
Simulates a simple market/model and visualizes equity, drawdown, and EV by edge bin.


In [ ]:
import numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, brier_score_loss, log_loss
rng = np.random.default_rng(123)
n = 4000
p_true = rng.beta(2.0,2.0,size=n)
logit = np.log(p_true/(1-p_true))
p_model = 1/(1+np.exp(-(logit + rng.normal(0,0.45,size=n))))
p_model = np.clip(p_model, 1e-4,1-1e-4)
p_base  = 1/(1+np.exp(-(logit + rng.normal(0,0.95,size=n))))
p_base  = np.clip(p_base,  1e-4,1-1e-4)
y = rng.binomial(1, p_true)
p_mkt = np.clip(p_true + rng.normal(0,0.08,size=n), 1e-3,1-1e-3)
price_mkt = 1.0 / p_mkt
price_fair_model = 1.0 / p_model
edge = (price_mkt - price_fair_model) / price_fair_model
def kelly_fraction(p, price, fmax=0.05):
    b = max(price-1.0, 0.0)
    q = 1.0 - p
    f = (b*p - q)/b if b>0 else 0.0
    return max(0.0, min(f, fmax))
stakes = np.array([kelly_fraction(p, pr, 0.05) for p, pr in zip(p_model, price_mkt)])
do_bet = edge >= 0.05
stakes = stakes * do_bet
wins = rng.binomial(1, p_true, size=n)
pl_steps = stakes * (wins * (price_mkt - 1.0) - (1 - wins))
equity = 1.0 + np.cumsum(pl_steps)
peak = np.maximum.accumulate(equity)
drawdown = (equity - peak) / peak
plt.figure(); plt.plot(equity); plt.title('Equity Curve'); plt.xlabel('Bet index'); plt.ylabel('Equity'); plt.grid(True,alpha=0.3); plt.show()
plt.figure(); plt.plot(drawdown); plt.title('Drawdown'); plt.xlabel('Bet index'); plt.ylabel('Drawdown'); plt.grid(True,alpha=0.3); plt.show()
bins = np.array([-0.2,-0.1,0.0,0.02,0.05,0.08,0.12,0.2,0.5])
ids = np.digitize(edge, bins) - 1
centers, ev_exp, ev_real = [], [], []
for b in range(len(bins)-1):
    m = ids==b
    if m.sum()==0: continue
    exp_ev = (p_model[m]*(price_mkt[m]-1.0) - (1.0 - p_model[m]))
    st = stakes[m]
    realized = np.where(st>0, (wins[m]*(price_mkt[m]-1.0) - (1 - wins[m])), np.nan)
    roi = realized[~np.isnan(realized)]
    ev_exp.append(np.nanmean(exp_ev))
    ev_real.append(np.nanmean(roi) if roi.size>0 else np.nan)
    centers.append(0.5*(bins[b]+bins[b+1]))
plt.figure(); plt.plot(centers, ev_exp, marker='o', label='Expected EV per bet');
plt.plot(centers, ev_real, marker='s', label='Realized ROI per bet');
plt.axvline(0.05, linestyle='--', label='Edge threshold');
plt.title('EV by Edge Bin'); plt.xlabel('Edge bin (center)'); plt.ylabel('Per-bet EV/ROI'); plt.grid(True,alpha=0.3); plt.legend(); plt.show()
